In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext()

Problem Statement 1:

In [7]:
#1. Read the text file and create a tupled rdd.

In [8]:
baseRDD = sc.textFile('Student_Dataset.txt')

In [9]:
type(baseRDD)

pyspark.rdd.RDD

In [11]:
student_data = baseRDD.map(lambda lst: (lst.split(','))).map(lambda x: (x[0], x[1], x[2], x[3], x[4]))
student_data.collect()

[('Mathew', 'science', 'grade-3', '45', '12'),
 ('Mathew', 'history', 'grade-2', '55', '13'),
 ('Mark', 'maths', 'grade-2', '23', '13'),
 ('Mark', 'science', 'grade-1', '76', '13'),
 ('John', 'history', 'grade-1', '14', '12'),
 ('John', 'maths', 'grade-2', '74', '13'),
 ('Lisa', 'science', 'grade-1', '24', '12'),
 ('Lisa', 'history', 'grade-3', '86', '13'),
 ('Andrew', 'maths', 'grade-1', '34', '13'),
 ('Andrew', 'science', 'grade-3', '26', '14'),
 ('Andrew', 'history', 'grade-1', '74', '12'),
 ('Mathew', 'science', 'grade-2', '55', '12'),
 ('Mathew', 'history', 'grade-2', '87', '12'),
 ('Mark', 'maths', 'grade-1', '92', '13'),
 ('Mark', 'science', 'grade-2', '12', '12'),
 ('John', 'history', 'grade-1', '67', '13'),
 ('John', 'maths', 'grade-1', '35', '11'),
 ('Lisa', 'science', 'grade-2', '24', '13'),
 ('Lisa', 'history', 'grade-2', '98', '15'),
 ('Andrew', 'maths', 'grade-1', '23', '16'),
 ('Andrew', 'science', 'grade-3', '44', '14'),
 ('Andrew', 'history', 'grade-2', '77', '11')]

In [12]:
type(student_data)

pyspark.rdd.PipelinedRDD

In [14]:
#2. Find the count of total number of rows present.
print("total number of records present: ",student_data.count())

total number of records present:  22


In [36]:
#3. What is the distinct number of subjects present in the entire school
print("distinct number of subjects present in the entire school: ",\
      student_data.map(lambda lst: (lst[1],'1'))\
         .reduceByKey(lambda amt1,amt2 : float(amt1)+float(amt2))\
         .collect())

distinct number of subjects present in the entire school:  [('science', 8.0), ('history', 8.0), ('maths', 6.0)]


In [37]:
#4. What is the count of the number of students in the school, whose name is Mathew and marks are 55
print("number of students in the school, whose name is Mathew and marks are 55: ",\
      student_data.map(lambda x: x if x[0]=='Mathew' and x[3]=='55' else 1).filter(lambda x: x!=1).count())

number of students in the school, whose name is Mathew and marks are 55:  2


Problem Statement 2:

In [39]:
#1. What is the count of students per grade in the school?
print("count of students per grade in the school: ",\
      student_data.map(lambda lst: (lst[2],'1'))\
         .reduceByKey(lambda amt1,amt2 : float(amt1)+float(amt2))\
         .collect())

count of students per grade in the school:  [('grade-1', 9.0), ('grade-2', 9.0), ('grade-3', 4.0)]


In [100]:
#2. Find the average of each student (Note - Mathew is grade-1, is different from Mathew in some other grade!)
print("count of students per grade in the school: ",\
      student_data.map(lambda lst: ((lst[0],lst[2]),lst[3]))\
         .reduceByKey(lambda amt1,amt2 : float(amt1)+float(amt2))\
         .collect())

count of students per grade in the school:  [(('Mark', 'grade-2'), 35.0), (('John', 'grade-1'), 116.0), (('Andrew', 'grade-2'), '77'), (('Lisa', 'grade-1'), '24'), (('Lisa', 'grade-3'), '86'), (('Mathew', 'grade-3'), '45'), (('Mark', 'grade-1'), 168.0), (('John', 'grade-2'), '74'), (('Andrew', 'grade-3'), 70.0), (('Andrew', 'grade-1'), 131.0), (('Lisa', 'grade-2'), 122.0), (('Mathew', 'grade-2'), 197.0)]


In [96]:
baseRDD = student_data.map(lambda lst: ((lst[0],lst[2]),lst[3]))
RDDmap = baseRDD.mapValues(lambda x: (x,1))
RDDreduce = RDDmap.reduceByKey(lambda x,y: (float(x._1) + float(y._1), float(x._2) + float(y._2)))
StudAvg = RDDreduce.mapValues(lambda sum,count:(1.0*sum)/count)

In [101]:
type(StudAvg)

pyspark.rdd.PipelinedRDD

In [109]:
#3. What is the average score of students in each subject across all grades?
baseRDD = student_data.map(lambda lst:((x.split(",")(0),x.split(",")(1)),x.split(",")(3).toInt))
RDDmap = baseRDD.mapValues(lambda x: (x,1))
RDDreduce = RDDmap.reduceByKey(lambda x,y: (x._1+y._1,x._2+y._2))
SubAvg = RDDreduce.mapValues(lambda sum,count: (1.0*sum)/count)

In [110]:
type(SubAvg)

pyspark.rdd.PipelinedRDD